In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.layers import LSTM

from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import h5py
import numpy as np
import random
import json
import sys

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 57
nb sequences: 200285


In [4]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
X.shape, y.shape

((200285, 40, 57), (200285, 57))

In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
model.fit(X, y,
          batch_size=256,
          epochs=40)

Epoch 1/40
200285/200285 [==============================] - 39s 196us/step - loss: 2.0642 - acc: 0.4010
Epoch 2/40
200285/200285 [==============================] - 37s 187us/step - loss: 1.6599 - acc: 0.5065
Epoch 3/40
200285/200285 [==============================] - 37s 186us/step - loss: 1.5461 - acc: 0.5362
Epoch 4/40
200285/200285 [==============================] - 38s 191us/step - loss: 1.4879 - acc: 0.5520
Epoch 5/40
200285/200285 [==============================] - 37s 187us/step - loss: 1.4498 - acc: 0.5620
Epoch 6/40
200285/200285 [==============================] - 39s 195us/step - loss: 1.4230 - acc: 0.5689
Epoch 7/40
200285/200285 [==============================] - 38s 189us/step - loss: 1.4030 - acc: 0.5742
Epoch 8/40
200285/200285 [==============================] - 37s 186us/step - loss: 1.3861 - acc: 0.5785
Epoch 9/40
200285/200285 [==============================] - 37s 186us/step - loss: 1.3729 - acc: 0.5818
Epoch 10/40
200285/200285 [==============================] - 37s

In [9]:
import os
os.makedirs('keras_js', exist_ok=True)
model.save('keras_js/nietzsche.h5')
with open('keras_js/chars.js', 'w') as fout:
    fout.write('maxlen = ' + str(maxlen) + '\n')
    fout.write('num_chars = ' + str(len(chars)) + '\n')
    fout.write('char_indices = ' + json.dumps(char_indices, indent=2) + '\n')
    fout.write('indices_char = ' + json.dumps(indices_char, indent=2) + '\n')